<h1><center>  新闻跟踪和分析 -  Nucleus API实例</center></h1>


<h1><center>  所有权及保密条款属SumUp Analytics所有</center></h1>


<h1><center>  免责声明和服务条款可通过 www.sumup.ai 获取 </center></h1> 

 


## 目标: 
-	开发一个工作流程以执行以下操作。
 - 识别和跟踪新闻/社交媒体中的某些主题
 - 从主要贡献者和外卖方面分析这些主题


## 数据:
-	新闻媒体RSS的集合
-   社交媒体供稿

    **Nucleus Datafeed自带200个新闻媒体RSS的内容**


## Nucleus APIs used:
-	数据集创建 API
 - 	*api_instance.post_upload_file(file, dataset)*
 - 	*nucleus_helper.import_files(api_instance, dataset, file_iters, processes=1)*

        nucleus_helper.import_files利用api_instance.post_upload_file并行执行来加速数据集的创建

-	主题建模 API
 - 	*api_instance.post_topic_api(payload)*


-	主题历史分析 API
 - 	*api_instance.post_topic_historical_analysis_api(payload)*


-	作者连接 API
 - 	*api_instance.post_author_connectivity_api(payload)*
 

-	主题摘要 API
 -  *api_instance.post_topic_summary_api(payload)*


## 方法:

### 1.	数据集准备
-	创建一个Nucleus数据集，其中包含所选历史时期内的所有相关文档

    

In [ ]:
import csv
import json
import nucleus_api.api.nucleus_api as nucleus_helper
import nucleus_api
from nucleus_api.rest import ApiException

configuration = nucleus_api.Configuration()
configuration.host = 'UPDATE-WITH-API-SERVER-HOSTNAME'
configuration.api_key['x-api-key'] = 'UPDATE-WITH-API-KEY'

# 创建API实例
api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

In [ ]:
# 利用自己的语料库
print('---- Case 1: you are using your own corpus, coming from a local folder ----')
folder = 'Twitter_feed'         
dataset = 'Twitter_feed'# str | 文件的目标数据集。

# 以递归方式从文件夹构建文件。 
# 其中每一项采用以下格式：
# {'filename': filename,   # 要上传的文件名。 需要
#  'metadata': {           # 该文件的元数据。 可选
#      'key1': val1,       # 只要名称，密钥就可以有任意名称
#      'key2': val2        # 包含字母数字（0-9 | a-z | A-Z）和下划线（_）
#   } 
# }
file_iter = []
for root, dirs, files in os.walk(folder):
    for file in files:
        #if Path(file).suffix == '.pdf': # .txt .doc .docx .rtf .html .csv also supported
        file_dict = {'filename': os.path.join(root, file),
                     'metadata': {'source': 'Tech Crunch',
                                  'author': 'Sarah Moore'
                                  'category': 'Media',
                                  'date': '2019-01-01'}}
        file_iter.append(file_dict)

file_props = nucleus_helper.upload_files(api_instance, dataset, file_iter, processes=4)
for fp in file_props:
    print(fp.filename, '(', fp.size, 'bytes) has been added to dataset', dataset)

    
# 利用Nucleus自带Feed 
print('---- Case 2: you are using an embedded datafeed ----')
dataset = 'sumup/rss_feed_ai'# 在Nucleus中导入自带数据。

-	对于该期间的给定日期，仅保留在所选回顾期间发布的文档的子集

**这可以直接在执行内容分析的API中完成，如下所示**



### 2.	内容分析和跟踪
-	识别并提取最近一段语库中的关键主题


-	跟踪这些关键主题到目前为止如何发展：相关性，感知 

In [ ]:
print('------------ Get topics + historical analysis ----------------')

payload = nucleus_api.TopicHistoryModel(dataset='Twitter_feed', 
                                    update_period='d', 
                                    query='',
                                    num_topics=20, 
                                    num_keywords=8,
                                    inc_step=1,
                                    period_start="2018-12-01 00:00:00",
                                    period_end="2019-01-01 00:00:00")
try:
    api_response = api_instance.post_topic_historical_analysis_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    print('Plotting historical metrics data...')
    historical_metrics = []
    for res in api_response.result:
        # 构建历史度量的图表列表
        historical_metrics.append({
            'topic'    : res.keywords,
            'time_stamps' : np.array(res.time_stamps),
            'strength' : np.array(res.strengths, dtype=np.float32),
            'consensus': np.array(res.consensuses, dtype=np.float32), 
            'sentiment': np.array(res.sentiments, dtype=np.float32)})

    selected_topics = range(len(historical_metrics)) 
    nucleus_helper.topic_charts_historical(historical_metrics, selected_topics, True)

-   对于与您相关的一些主题，请列出最近的主要内容（摘要，最佳来源）

In [ ]:
print('------------- Get the summaries of recent topics in your feed --------------')

payload = nucleus_api.TopicSummaryModel(dataset='Twitter_feed',                         
                            query='',                       
                            num_topics=20, 
                            num_keywords=8,
                            period_start="2018-12-31 00:00:00",
                            period_end="2019-01-01 00:00:00")
try:
    api_response = api_instance.post_topic_summary_api(payload)        
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:    
    for res in api_response.result:
        print('Topic', i, 'summary:')
        print('    Keywords:', res.topic)
        for j in range(len(res.summary)):
            print(res.summary[j])
            print('    Document ID:', res.summary[j].sourceid)
            print('        Title:', res.summary[j].title)
            print('        Sentences:', res.summary[j].sentences)
            print('        Author:', res.summary[j].attribute['author'])
            print('        Time:', datetime.datetime.fromtimestamp(float(res.summary[j].attribute['time'])))   

-   您可以通过利用作者连接性分析深入了解一些有影响力的人或有趣的新兴贡献者：

根据他们参与的主题和动机，谁与那个人最相似

In [ ]:
print('----------------- Get author connectivity -------------------')

payload = nucleus_api.AuthorConnection(dataset='Twitter_feed', 
                                        target_author='Yann LeCun', 
                                        query='',
                                        period_start="2018-12-31 00:00:00",
                                        period_end="2019-01-01 00:00:00")
try:
    api_response = api_instance.post_author_connectivity_api(payload)    
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:
    print('Mainstream connections:')
    for mc in api_response.result.mainstream_connections:
        print('    Topic:', mc.keywords)
        print('    Authors:', " ".join(str(x) for x in mc.authors))

    print('Niche connections:')
    for nc in api_response.result.niche_connections:
        print('    Topic:', nc.keywords)
        print('    Authors:', " ".join(str(x) for x in nc.authors))  

-	接下来，我们将讨论如何利用用户可用的不同参数来优化内容分析

### 3.	微调

#### a.	量身定制主题

-   使用主题历史分析API中的`custom_stop_words`参数排除无关的关键字/主题以定制内容分析


-	提取语库中的关键主题并打印其关键字

In [ ]:
print('------------- Get the recent topics in your feed --------------')

payload = nucleus_api.Topics(dataset='Twitter_feed',                         
                            query='',                       
                            num_topics=20, 
                            num_keywords=8,
                            period_start="2018-12-31 00:00:00",
                            period_end="2019-01-01 00:00:00")
try:
    api_response = api_instance.post_topic_api(payload)        
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:    
    for i, res in enumerate(api_response.result.topics):
        print('Topic', i, ' keywords: ', res.keywords)    
        print('---------------')

然后，您可以通过创建`custom_stop_words`变量来定制内容分析。 如下所示，将初始化变量传递到第2节主代码的有效负载中：

In [ ]:
custom_stop_words = ["supervised learning", "training"] # str | 停用词列表。 （可选）

#### b.	将内容分析集中在某些主题上
如果您决定关注内容分析，例如深度学习主题，只需将第2节主代码中的查询变量替换为：

In [ ]:
query = '(deep-learning OR LSTM OR RNN OR Neural network)' # str | Fulltext query, using mysql MATCH boolean query format. Example: "(word1 OR word2) AND (word3 OR word4)" (optional)

#### c.	探索文档类型，回顾期，提取的主题数量的影响
**period_starts**: 您可以使用不同的回顾期执行内容分析。 您可以根据目标在相关时间范围内对数据进行切片。

**Document types**: 您可以根据来源类型（例如，不同语言，来自学术界，私营部门或独立个人的贡献者）来调查主题及其随时间的演变如何变化。

 - 创建一个变量`metadata_selection`并将其传递给有效负载（如果使用您的文档或中央银行订阅源，新闻媒体RSS订阅源没有可以选择的元数据）。
 
 - 在整个语库的子集上重新运行第2节的主要代码。

In [ ]:
metadata_selection = {"category": "Academia"}   # str | json object of {\"metadata_field\":[\"selected_values\"]} (optional)

### 4.	下一步
-	可能的扩展：构建一个内部BI报告，提取主要主题和关键内容


-	可能的扩展：比较内部和外部的专家团队讨论，激发市场竞争性


版权(c) 2019年 SumUp Analytics 公司 版权所有。 

通知：所有信息均属于SumUp Analytics Inc公司及其供应商的财产。 本合同所包含的知识产权和技术概念属于SumUp Analytics Inc.及其供应商的专利，可由美国和外国专利、在工艺中的专利以及受贸易秘密或版权法保护的专利涵盖。 

除非得到SumUp Analytics公司的事先书面批准，否则严禁传播此类信息或复制此材料。 